## Task 1: generate simulated data with deletions

We are going to generate sequences with motif: ATATTCA and GTACTGC.

They will either appear together with a nulcleotid distance of 10-fold from each other: 10, 20, 30, .... nucleotides apart. (where the deletion don't count), but there can be deletions inside.

Or only one of them will appear.

And all the other part of the sequences will be randomly generated with deletions.

And then we will add "-" tokens for the deletion. Should we have deletion in the motif too? 

In [7]:
from fundemental_classes.dna_generator import generate_dataset

n=10000
length=150
deletions = 0.1
no_del_seqs = 0.25
gaps = list(range(10, 101, 10))

dataset = generate_dataset(n=n, length=length)

def write_fasta(dataset, filepath):
    with open(filepath, "w") as f:
        for i, ex in enumerate(dataset, start=1):
            header = (
                f">seq{i:04d}"
                f"|label={ex.label}"
                f"|posAmotif={ex.pos_a}"
                f"|posBmotif={ex.pos_b}"
                f"|gaplength={ex.gap}"
                f"|deletions={ex.deletions}"
            )
            f.write(header + "\n")
            f.write(ex.seq + "\n")
            

write_fasta(dataset, f"simulated_sequences/augumentedModified_sequence_size{n}_length{length}_deletions{deletions}_nodeletionseq{no_del_seqs}.fasta")